In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown, display

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from skopt import BayesSearchCV
from skopt.space import Real, Integer

Installation erfolgreich!


In [72]:
# Datei öffnen und lesen
with open('data_description.txt', 'r') as file:
    content = file.read()

# Inhalt als Markdown anzeigen
display(Markdown(content))

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density
	
LotFrontage: Linear feet of street connected to property

LotArea: Lot size in square feet

Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved
       	
Alley: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access
		
LotShape: General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular
       
LandContour: Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to building
       HLS	Hillside - Significant slope from side to side
       Low	Depression
		
Utilities: Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)	
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only	
	
LotConfig: Lot configuration

       Inside	Inside lot
       Corner	Corner lot
       CulDSac	Cul-de-sac
       FR2	Frontage on 2 sides of property
       FR3	Frontage on 3 sides of property
	
LandSlope: Slope of property
		
       Gtl	Gentle slope
       Mod	Moderate Slope	
       Sev	Severe Slope
	
Neighborhood: Physical locations within Ames city limits

       Blmngtn	Bloomington Heights
       Blueste	Bluestem
       BrDale	Briardale
       BrkSide	Brookside
       ClearCr	Clear Creek
       CollgCr	College Creek
       Crawfor	Crawford
       Edwards	Edwards
       Gilbert	Gilbert
       IDOTRR	Iowa DOT and Rail Road
       MeadowV	Meadow Village
       Mitchel	Mitchell
       Names	North Ames
       NoRidge	Northridge
       NPkVill	Northpark Villa
       NridgHt	Northridge Heights
       NWAmes	Northwest Ames
       OldTown	Old Town
       SWISU	South & West of Iowa State University
       Sawyer	Sawyer
       SawyerW	Sawyer West
       Somerst	Somerset
       StoneBr	Stone Brook
       Timber	Timberland
       Veenker	Veenker
			
Condition1: Proximity to various conditions
	
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
	
Condition2: Proximity to various conditions (if more than one is present)
		
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
	
BldgType: Type of dwelling
		
       1Fam	Single-family Detached	
       2FmCon	Two-family Conversion; originally built as one-family dwelling
       Duplx	Duplex
       TwnhsE	Townhouse End Unit
       TwnhsI	Townhouse Inside Unit
	
HouseStyle: Style of dwelling
	
       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level
	
OverallQual: Rates the overall material and finish of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor
	
OverallCond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor
		
YearBuilt: Original construction date

YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)

RoofStyle: Type of roof

       Flat	Flat
       Gable	Gable
       Gambrel	Gabrel (Barn)
       Hip	Hip
       Mansard	Mansard
       Shed	Shed
		
RoofMatl: Roof material

       ClyTile	Clay or Tile
       CompShg	Standard (Composite) Shingle
       Membran	Membrane
       Metal	Metal
       Roll	Roll
       Tar&Grv	Gravel & Tar
       WdShake	Wood Shakes
       WdShngl	Wood Shingles
		
Exterior1st: Exterior covering on house

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast	
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
	
Exterior2nd: Exterior covering on house (if more than one material)

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
	
MasVnrType: Masonry veneer type

       BrkCmn	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       None	None
       Stone	Stone
	
MasVnrArea: Masonry veneer area in square feet

ExterQual: Evaluates the quality of the material on the exterior 
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
ExterCond: Evaluates the present condition of the material on the exterior
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
Foundation: Type of foundation
		
       BrkTil	Brick & Tile
       CBlock	Cinder Block
       PConc	Poured Contrete	
       Slab	Slab
       Stone	Stone
       Wood	Wood
		
BsmtQual: Evaluates the height of the basement

       Ex	Excellent (100+ inches)	
       Gd	Good (90-99 inches)
       TA	Typical (80-89 inches)
       Fa	Fair (70-79 inches)
       Po	Poor (<70 inches
       NA	No Basement
		
BsmtCond: Evaluates the general condition of the basement

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness
       NA	No Basement
	
BsmtExposure: Refers to walkout or garden level walls

       Gd	Good Exposure
       Av	Average Exposure (split levels or foyers typically score average or above)	
       Mn	Mimimum Exposure
       No	No Exposure
       NA	No Basement
	
BsmtFinType1: Rating of basement finished area

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement
		
BsmtFinSF1: Type 1 finished square feet

BsmtFinType2: Rating of basement finished area (if multiple types)

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement

BsmtFinSF2: Type 2 finished square feet

BsmtUnfSF: Unfinished square feet of basement area

TotalBsmtSF: Total square feet of basement area

Heating: Type of heating
		
       Floor	Floor Furnace
       GasA	Gas forced warm air furnace
       GasW	Gas hot water or steam heat
       Grav	Gravity furnace	
       OthW	Hot water or steam heat other than gas
       Wall	Wall furnace
		
HeatingQC: Heating quality and condition

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
CentralAir: Central air conditioning

       N	No
       Y	Yes
		
Electrical: Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed
		
1stFlrSF: First Floor square feet
 
2ndFlrSF: Second floor square feet

LowQualFinSF: Low quality finished square feet (all floors)

GrLivArea: Above grade (ground) living area square feet

BsmtFullBath: Basement full bathrooms

BsmtHalfBath: Basement half bathrooms

FullBath: Full bathrooms above grade

HalfBath: Half baths above grade

Bedroom: Bedrooms above grade (does NOT include basement bedrooms)

Kitchen: Kitchens above grade

KitchenQual: Kitchen quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       	
TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)

Functional: Home functionality (Assume typical unless deductions are warranted)

       Typ	Typical Functionality
       Min1	Minor Deductions 1
       Min2	Minor Deductions 2
       Mod	Moderate Deductions
       Maj1	Major Deductions 1
       Maj2	Major Deductions 2
       Sev	Severely Damaged
       Sal	Salvage only
		
Fireplaces: Number of fireplaces

FireplaceQu: Fireplace quality

       Ex	Excellent - Exceptional Masonry Fireplace
       Gd	Good - Masonry Fireplace in main level
       TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
       Fa	Fair - Prefabricated Fireplace in basement
       Po	Poor - Ben Franklin Stove
       NA	No Fireplace
		
GarageType: Garage location
		
       2Types	More than one type of garage
       Attchd	Attached to home
       Basment	Basement Garage
       BuiltIn	Built-In (Garage part of house - typically has room above garage)
       CarPort	Car Port
       Detchd	Detached from home
       NA	No Garage
		
GarageYrBlt: Year garage was built
		
GarageFinish: Interior finish of the garage

       Fin	Finished
       RFn	Rough Finished	
       Unf	Unfinished
       NA	No Garage
		
GarageCars: Size of garage in car capacity

GarageArea: Size of garage in square feet

GarageQual: Garage quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage
		
GarageCond: Garage condition

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage
		
PavedDrive: Paved driveway

       Y	Paved 
       P	Partial Pavement
       N	Dirt/Gravel
		
WoodDeckSF: Wood deck area in square feet

OpenPorchSF: Open porch area in square feet

EnclosedPorch: Enclosed porch area in square feet

3SsnPorch: Three season porch area in square feet

ScreenPorch: Screen porch area in square feet

PoolArea: Pool area in square feet

PoolQC: Pool quality
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool
		
Fence: Fence quality
		
       GdPrv	Good Privacy
       MnPrv	Minimum Privacy
       GdWo	Good Wood
       MnWw	Minimum Wood/Wire
       NA	No Fence
	
MiscFeature: Miscellaneous feature not covered in other categories
		
       Elev	Elevator
       Gar2	2nd Garage (if not described in garage section)
       Othr	Other
       Shed	Shed (over 100 SF)
       TenC	Tennis Court
       NA	None
		
MiscVal: $Value of miscellaneous feature

MoSold: Month Sold (MM)

YrSold: Year Sold (YYYY)

SaleType: Type of sale
		
       WD 	Warranty Deed - Conventional
       CWD	Warranty Deed - Cash
       VWD	Warranty Deed - VA Loan
       New	Home just constructed and sold
       COD	Court Officer Deed/Estate
       Con	Contract 15% Down payment regular terms
       ConLw	Contract Low Down payment and low interest
       ConLI	Contract Low Interest
       ConLD	Contract Low Down
       Oth	Other
		
SaleCondition: Condition of sale

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)


In [73]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [74]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [75]:
missing_values_count = train.isnull().sum()
missing_values_percent = (train.isnull().sum() / len(train)) * 100

# Umwandeln des Ergebnisses in ein DataFrame
missing_data_df = pd.DataFrame({
    'Missing Values': missing_values_count,
    'Percent Missing': missing_values_percent
})

missing_data_df = missing_data_df[missing_data_df['Missing Values'] > 0]
missing_data_df

,Missing Values,Percent Missing
LotFrontage,259,17.739726
Alley,1369,93.767123
MasVnrType,872,59.726027
MasVnrArea,8,0.547945
BsmtQual,37,2.534247
BsmtCond,37,2.534247
BsmtExposure,38,2.602740
BsmtFinType1,37,2.534247
BsmtFinType2,38,2.602740
Electrical,1,0.068493


In [76]:
unique_values_count = train['BsmtQual'].value_counts()

# Ausgabe des Ergebnisses
print(unique_values_count)

BsmtQual
TA    649
Gd    618
Ex    121
Fa     35
Name: count, dtype: int64


In [77]:
data = train.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'])

for column in data.select_dtypes(include=['float64', 'int64']).columns:
    if data[column].isnull().mean() > 0:
        data[column] = data[column].fillna(data[column].mean())

for column in data.select_dtypes(include=['object']).columns:
    if data[column].isnull().mean() > 0:
        data[column] = data[column].fillna(data[column].mode()[0])

data = pd.get_dummies(data, drop_first=True)

missing_values_after = data.isnull().sum().sum()
print(f"Missing values after preprocessing: {missing_values_after}")

Missing values after preprocessing: 0


In [78]:
# MasVnrArea -> mean

In [79]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Extrahieren der Merkmale und Zielvariable
X = data.drop(columns=['SalePrice'])
y = data['SalePrice']

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialisieren und Trainieren des XGBoost Regressors
xg_reg = xgb.XGBRegressor(objective='reg:squarederror',
                          colsample_bytree=0.3,
                          learning_rate=0.1,
                          max_depth=5,
                          alpha=10,
                          n_estimators=500)
xg_reg.fit(X_train, y_train)

# Vorhersagen und Berechnen des RMSE
y_pred = xg_reg.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"XGBoost RMSE: {rmse:.2f}")


XGBoost RMSE: 25913.94


In [80]:
from sklearn.ensemble import RandomForestRegressor

# Initialisieren und Trainieren des RandomForestRegressors
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(X_train, y_train)

# Vorhersagen und Berechnen des RMSE
y_pred_rf = rf_reg.predict(X_test)
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
print(f"Random Forest RMSE: {rmse_rf:.2f}")


Random Forest RMSE: 28711.87


In [81]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Angenommen, 'data' ist Ihr DataFrame mit den Features und der Zielvariable 'SalePrice'
# Extrahieren der Merkmale und Zielvariable
X = data.drop(columns=['SalePrice'])
y = data['SalePrice']

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sicherstellen, dass alle Daten numerisch sind und in numpy-Arrays konvertieren
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)

# Initialisieren des Keras-Modells
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
])

# Kompilieren des Modells
model.compile(optimizer='adam', loss='mean_squared_error')

# Trainieren des Modells
model.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=1)

# Vorhersagen und Berechnen des RMSE
y_pred_nn = model.predict(X_test)
rmse_nn = mean_squared_error(y_test, y_pred_nn, squared=False)
print(f"Keras NN RMSE: {rmse_nn:.2f}")


Epoch 1/100
30/30 [==============================] - 0s 3ms/step - loss: 36435611648.0000 - val_loss: 31634827264.0000
Epoch 2/100
30/30 [==============================] - 0s 1ms/step - loss: 25729685504.0000 - val_loss: 15305826304.0000
Epoch 3/100
30/30 [==============================] - 0s 994us/step - loss: 11198713856.0000 - val_loss: 4455456768.0000
Epoch 4/100
30/30 [==============================] - 0s 963us/step - loss: 7455336960.0000 - val_loss: 3460407040.0000
Epoch 5/100
30/30 [==============================] - 0s 999us/step - loss: 6308779520.0000 - val_loss: 3337753344.0000
Epoch 6/100
30/30 [==============================] - 0s 982us/step - loss: 5449936896.0000 - val_loss: 2950224896.0000
Epoch 7/100
30/30 [==============================] - 0s 1ms/step - loss: 4702369280.0000 - val_loss: 2782104320.0000
Epoch 8/100
30/30 [==============================] - 0s 1ms/step - loss: 4240712704.0000 - val_loss: 2676487936.0000
Epoch 9/100
30/30 [==============================] 

In [82]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Angenommen, 'data' ist Ihr DataFrame mit den Features und der Zielvariable 'SalePrice'
# Extrahieren der Merkmale und Zielvariable
X = data.drop(columns=['SalePrice'])
y = data['SalePrice']

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sicherstellen, dass alle Daten numerisch sind und in numpy-Arrays konvertieren
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)

# Initialisieren der Modelle
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "XGBoost Regressor": xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1, max_depth=5, alpha=10, n_estimators=100, random_state=42)
}

# Trainieren und Evaluieren der Modelle
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(f"{name} RMSE: {rmse:.2f}")


Linear Regression RMSE: 61017.14
Ridge Regression RMSE: 30249.00
Lasso Regression RMSE: 51803.72
Decision Tree Regressor RMSE: 43192.41


/Users/linuszarse/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.316e+11, tolerance: 6.967e+08
  model = cd_fast.enet_coordinate_descent(


Random Forest Regressor RMSE: 28711.87
Gradient Boosting Regressor RMSE: 27878.01
XGBoost Regressor RMSE: 26023.49


In [83]:
# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sicherstellen, dass alle Daten numerisch sind und in numpy-Arrays konvertieren
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)

In [84]:
ridge_params = {
    'alpha': [0.1, 1.0, 10.0, 100.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

ridge = Ridge()
ridge_grid = GridSearchCV(ridge, ridge_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
ridge_grid.fit(X_train, y_train)

best_ridge = ridge_grid.best_estimator_
ridge_rmse = mean_squared_error(y_test, best_ridge.predict(X_test), squared=False)
print(f"Best Ridge Regression RMSE: {ridge_rmse:.2f}")


/Users/linuszarse/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.97269e-13): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/linuszarse/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.97269e-13): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/linuszarse/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/linuszarse/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/linuszarse/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter wa

Best Ridge Regression RMSE: 30705.62


In [85]:
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestRegressor(random_state=42)
rf_grid = RandomizedSearchCV(rf, rf_params, cv=5, scoring='neg_mean_squared_error', n_iter=10, n_jobs=-1, random_state=42)
rf_grid.fit(X_train, y_train)

best_rf = rf_grid.best_estimator_
rf_rmse = mean_squared_error(y_test, best_rf.predict(X_test), squared=False)
print(f"Best Random Forest RMSE: {rf_rmse:.2f}")


Best Random Forest RMSE: 28899.30


In [86]:
gb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

gb = GradientBoostingRegressor(random_state=42)
gb_grid = RandomizedSearchCV(gb, gb_params, cv=5, scoring='neg_mean_squared_error', n_iter=10, n_jobs=-1, random_state=42)
gb_grid.fit(X_train, y_train)

best_gb = gb_grid.best_estimator_
gb_rmse = mean_squared_error(y_test, best_gb.predict(X_test), squared=False)
print(f"Best Gradient Boosting RMSE: {gb_rmse:.2f}")


Best Gradient Boosting RMSE: 26143.16


In [87]:
xgb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
}

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_grid = RandomizedSearchCV(xgb_reg, xgb_params, cv=5, scoring='neg_mean_squared_error', n_iter=10, n_jobs=-1, random_state=42)
xgb_grid.fit(X_train, y_train)

best_xgb = xgb_grid.best_estimator_
xgb_rmse = mean_squared_error(y_test, best_xgb.predict(X_test), squared=False)
print(f"Best XGBoost RMSE: {xgb_rmse:.2f}")


Best XGBoost RMSE: 25682.01


In [88]:
print(f"Best Ridge Regression RMSE: {ridge_rmse:.2f}")
print(f"Best Random Forest RMSE: {rf_rmse:.2f}")
print(f"Best Gradient Boosting RMSE: {gb_rmse:.2f}")
print(f"Best XGBoost RMSE: {xgb_rmse:.2f}")


Best Ridge Regression RMSE: 30705.62
Best Random Forest RMSE: 28899.30
Best Gradient Boosting RMSE: 26143.16
Best XGBoost RMSE: 25682.01


In [89]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# Angenommen, 'data' ist Ihr DataFrame mit den Features und der Zielvariable 'SalePrice'
# Extrahieren der Merkmale und Zielvariable
X = data.drop(columns=['SalePrice'])
y = data['SalePrice']

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sicherstellen, dass alle Daten numerisch sind und in numpy-Arrays konvertieren
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [93]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna


def objective_ridge(trial):
    alpha = trial.suggest_loguniform('alpha', 0.1, 100.0)
    solver = trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])
    
    model = Ridge(alpha=alpha, solver=solver)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    return rmse

study_ridge = optuna.create_study(direction='minimize')
study_ridge.optimize(objective_ridge, n_trials=50, n_jobs=-1)

best_params_ridge = study_ridge.best_params
best_ridge = Ridge(**best_params_ridge)
best_ridge.fit(X_train, y_train)
ridge_rmse = mean_squared_error(y_test, best_ridge.predict(X_test), squared=False)
print(f"Best Ridge Regression RMSE: {ridge_rmse:.2f}")


[I 2024-06-15 20:29:09,736] A new study created in memory with name: no-name-88c7e09c-222a-415d-aaba-50e25d5e5ac1
/var/folders/2b/c643wm_d7xl9868bh1t2r6n80000gn/T/ipykernel_41526/180743021.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 0.1, 100.0)
/var/folders/2b/c643wm_d7xl9868bh1t2r6n80000gn/T/ipykernel_41526/180743021.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 0.1, 100.0)
[I 2024-06-15 20:29:11,470] Trial 2 finished with value: 31116.81640625 and parameters: {'alpha': 0.18713254991512904, 'solver': 'svd'}. Best is trial 2 with value: 31116.81640625.
[I 2024-06-15 20:29

Best Ridge Regression RMSE: 30239.68


In [94]:
best_params_ridge

{'alpha': 1.2990971084630494, 'solver': 'cholesky'}

In [98]:
# Definieren der Parameter für Bayesian Optimization
gb_params = {
    'n_estimators': Integer(50, 300),
    'learning_rate': Real(0.01, 0.1, prior='log-uniform'),
    'max_depth': Integer(3, 10),
    'subsample': Real(0.7, 1.0),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 4)
}

# Initialisieren des Gradient Boosting Regressors
gb = GradientBoostingRegressor(random_state=42)

# Bayesian Optimization mit BayesSearchCV
gb_opt = BayesSearchCV(gb, gb_params, n_iter=32, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
gb_opt.fit(X_train, y_train)

# Bestes Modell und RMSE berechnen
best_gb = gb_opt.best_estimator_
gb_rmse = mean_squared_error(y_test, best_gb.predict(X_test), squared=False)
print(f"Best Gradient Boosting RMSE: {gb_rmse:.2f}")


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [101]:
print(f"Best Ridge Regression RMSE: {ridge_rmse:.2f}")
print(f"Best Random Forest RMSE: {rf_rmse:.2f}")
print(f"Best Gradient Boosting RMSE: {gb_rmse:.2f}")
print(f"Best XGBoost RMSE: {xgb_rmse:.2f}")


Best Ridge Regression RMSE: 30239.68
Best Random Forest RMSE: 28899.30
Best Gradient Boosting RMSE: 26143.16
Best XGBoost RMSE: 25682.01


In [102]:
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)
    
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    return rmse

study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(objective_rf, n_trials=50, n_jobs=-1)
best_params_rf = study_rf.best_params

best_rf = RandomForestRegressor(
    n_estimators=best_params_rf['n_estimators'],
    max_depth=best_params_rf['max_depth'],
    min_samples_split=best_params_rf['min_samples_split'],
    min_samples_leaf=best_params_rf['min_samples_leaf'],
    random_state=42
)
best_rf.fit(X_train, y_train)
rf_rmse = mean_squared_error(y_test, best_rf.predict(X_test), squared=False)
print(f"Best Random Forest RMSE: {rf_rmse:.2f}")


[I 2024-06-15 20:56:27,377] A new study created in memory with name: no-name-9a2f4b7e-443b-4a31-b02a-2d193f24d375
[I 2024-06-15 20:56:29,353] Trial 4 finished with value: 29728.652976538604 and parameters: {'n_estimators': 130, 'max_depth': 9, 'min_samples_split': 9, 'min_samples_leaf': 2}. Best is trial 4 with value: 29728.652976538604.
[I 2024-06-15 20:56:29,934] Trial 2 finished with value: 30751.29441319724 and parameters: {'n_estimators': 160, 'max_depth': 48, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 4 with value: 29728.652976538604.
[I 2024-06-15 20:56:29,988] Trial 0 finished with value: 31498.447030164287 and parameters: {'n_estimators': 202, 'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 4 with value: 29728.652976538604.
[I 2024-06-15 20:56:30,323] Trial 7 finished with value: 29508.267494398104 and parameters: {'n_estimators': 194, 'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 2}. Best is trial 7 with value: 2

Best Random Forest RMSE: 28451.60


In [103]:
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)
    
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    return rmse

study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(objective_rf, n_trials=50, n_jobs=-1)
best_params_rf = study_rf.best_params

best_rf = RandomForestRegressor(
    n_estimators=best_params_rf['n_estimators'],
    max_depth=best_params_rf['max_depth'],
    min_samples_split=best_params_rf['min_samples_split'],
    min_samples_leaf=best_params_rf['min_samples_leaf'],
    random_state=42
)
best_rf.fit(X_train, y_train)
rf_rmse = mean_squared_error(y_test, best_rf.predict(X_test), squared=False)
print(f"Best Random Forest RMSE: {rf_rmse:.2f}")


[I 2024-06-15 20:56:49,893] A new study created in memory with name: no-name-ba8bcf13-02ac-40b4-91ef-27664c4d6115
[I 2024-06-15 20:56:50,992] Trial 3 finished with value: 29485.683126249038 and parameters: {'n_estimators': 64, 'max_depth': 18, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 3 with value: 29485.683126249038.
[I 2024-06-15 20:56:51,245] Trial 4 finished with value: 31678.991522639375 and parameters: {'n_estimators': 120, 'max_depth': 6, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 3 with value: 29485.683126249038.
[I 2024-06-15 20:56:51,438] Trial 5 finished with value: 29577.23867106106 and parameters: {'n_estimators': 94, 'max_depth': 42, 'min_samples_split': 7, 'min_samples_leaf': 1}. Best is trial 3 with value: 29485.683126249038.
[I 2024-06-15 20:56:52,016] Trial 0 finished with value: 29774.61650933409 and parameters: {'n_estimators': 136, 'max_depth': 36, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 3 with value: 294

Best Random Forest RMSE: 28722.55


In [104]:
def objective_gb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.1)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    subsample = trial.suggest_uniform('subsample', 0.7, 1.0)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)
    
    model = GradientBoostingRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    return rmse

study_gb = optuna.create_study(direction='minimize')
study_gb.optimize(objective_gb, n_trials=50, n_jobs=-1)
best_params_gb = study_gb.best_params

best_gb = GradientBoostingRegressor(
    n_estimators=best_params_gb['n_estimators'],
    learning_rate=best_params_gb['learning_rate'],
    max_depth=best_params_gb['max_depth'],
    subsample=best_params_gb['subsample'],
    min_samples_split=best_params_gb['min_samples_split'],
    min_samples_leaf=best_params_gb['min_samples_leaf'],
    random_state=42
)
best_gb.fit(X_train, y_train)
gb_rmse = mean_squared_error(y_test, best_gb.predict(X_test), squared=False)
print(f"Best Gradient Boosting RMSE: {gb_rmse:.2f}")


[I 2024-06-15 20:57:11,103] A new study created in memory with name: no-name-3abde58b-ccb0-46d9-94de-6ffc3b52ba57
/var/folders/2b/c643wm_d7xl9868bh1t2r6n80000gn/T/ipykernel_41526/3594107221.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.1)
/var/folders/2b/c643wm_d7xl9868bh1t2r6n80000gn/T/ipykernel_41526/3594107221.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.7, 1.0)
/var/folders/2b/c643wm_d7xl9868bh1t2r6n80000gn/T/ipykernel_41526/3594107221.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://gith

Best Gradient Boosting RMSE: 24802.52


In [119]:
def objective_gb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 800)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.1)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    subsample = trial.suggest_uniform('subsample', 0.7, 1.0)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)
    
    model = GradientBoostingRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    return rmse

study_gb = optuna.create_study(direction='minimize')
study_gb.optimize(objective_gb, n_trials=50, n_jobs=-1)
best_params_gb = study_gb.best_params

best_gb = GradientBoostingRegressor(
    n_estimators=best_params_gb['n_estimators'],
    learning_rate=best_params_gb['learning_rate'],
    max_depth=best_params_gb['max_depth'],
    subsample=best_params_gb['subsample'],
    min_samples_split=best_params_gb['min_samples_split'],
    min_samples_leaf=best_params_gb['min_samples_leaf'],
    random_state=42
)
best_gb.fit(X_train, y_train)
gb_rmse = mean_squared_error(y_test, best_gb.predict(X_test), squared=False)
print(f"Best Gradient Boosting RMSE: {gb_rmse:.2f}")


[I 2024-06-15 21:13:33,551] A new study created in memory with name: no-name-3ce1d8c0-cd3a-44ad-9d85-7663a2a31efc
/var/folders/2b/c643wm_d7xl9868bh1t2r6n80000gn/T/ipykernel_41526/3938567086.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.1)
/var/folders/2b/c643wm_d7xl9868bh1t2r6n80000gn/T/ipykernel_41526/3938567086.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.7, 1.0)
[I 2024-06-15 21:13:35,125] Trial 2 finished with value: 35926.3218633755 and parameters: {'n_estimators': 107, 'learning_rate': 0.014755484844178908, 'max_depth': 5, 'subsample': 0.9597593532168183, 

Best Gradient Boosting RMSE: 22648.53


In [120]:
best_params_gb

{'n_estimators': 712,
 'learning_rate': 0.07052549739590212,
 'max_depth': 5,
 'subsample': 0.7491641541674455,
 'min_samples_split': 5,
 'min_samples_leaf': 4}

In [106]:
def objective_xgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.1)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    subsample = trial.suggest_uniform('subsample', 0.7, 1.0)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.7, 1.0)
    
    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        objective='reg:squarederror',
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    return rmse

study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=50, n_jobs=-1)
best_params_xgb = study_xgb.best_params

best_xgb = xgb.XGBRegressor(
    n_estimators=best_params_xgb['n_estimators'],
    learning_rate=best_params_xgb['learning_rate'],
    max_depth=best_params_xgb['max_depth'],
    subsample=best_params_xgb['subsample'],
    colsample_bytree=best_params_xgb['colsample_bytree'],
    objective='reg:squarederror',
    random_state=42
)
best_xgb.fit(X_train, y_train)
xgb_rmse = mean_squared_error(y_test, best_xgb.predict(X_test), squared=False)
print(f"Best XGBoost RMSE: {xgb_rmse:.2f}")


[I 2024-06-15 20:57:46,762] A new study created in memory with name: no-name-3b6855c1-35d7-497a-8e94-58cfefed30c6
/var/folders/2b/c643wm_d7xl9868bh1t2r6n80000gn/T/ipykernel_41526/640904044.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.1)
/var/folders/2b/c643wm_d7xl9868bh1t2r6n80000gn/T/ipykernel_41526/640904044.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.7, 1.0)
/var/folders/2b/c643wm_d7xl9868bh1t2r6n80000gn/T/ipykernel_41526/640904044.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.

Best XGBoost RMSE: 24060.00


In [107]:
print(f"Best Ridge Regression RMSE: {ridge_rmse:.2f}")
print(f"Best Random Forest RMSE: {rf_rmse:.2f}")
print(f"Best Gradient Boosting RMSE: {gb_rmse:.2f}")
print(f"Best XGBoost RMSE: {xgb_rmse:.2f}")


Best Ridge Regression RMSE: 30239.68
Best Random Forest RMSE: 28722.55
Best Gradient Boosting RMSE: 23895.52
Best XGBoost RMSE: 24060.00


***
### Anwenden Models

In [110]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# Behandeln der fehlenden Werte im Testdatensatz
test_data = test.copy()

for column in test_data.select_dtypes(include=['float64', 'int64']).columns:
    if test_data[column].isnull().mean() > 0:
        test_data[column] = test_data[column].fillna(test_data[column].mean())

for column in test_data.select_dtypes(include=['object']).columns:
    if test_data[column].isnull().mean() > 0:
        test_data[column] = test_data[column].fillna('Missing')

test_data = pd.get_dummies(test_data, drop_first=True)

missing_cols = set(X_train.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0
test_data = test_data[X_train.columns]

X_test_final = test_data.astype(np.float32).values

# Vorhersagen mit dem XGBoost-Modell
y_pred_xgboost = best_xgb.predict(X_test_final)

# Vorhersagen mit dem finetuned Keras-Modell
y_pred_nn_final = model_finetune.predict(X_test_final)

# Sicherstellen, dass 'Id' in den ursprünglichen Testdaten vorhanden ist
test_ids = test['Id'].values

NameError: name 'model_finetune' is not defined

In [111]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# Vorbereitungen zur Behandlung der fehlenden Werte im Testdatensatz
test_data = test.copy()

for column in test_data.select_dtypes(include=['float64', 'int64']).columns:
    if test_data[column].isnull().mean() > 0:
        test_data[column] = test_data[column].fillna(test_data[column].mean())

for column in test_data.select_dtypes(include=['object']).columns:
    if test_data[column].isnull().mean() > 0:
        test_data[column] = test_data[column].fillna('Missing')

test_data = pd.get_dummies(test_data, drop_first=True)

missing_cols = set(X_train.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0
test_data = test_data[X_train.columns]

X_test_final = test_data.astype(np.float32).values

# Sicherstellen, dass 'Id' in den ursprünglichen Testdaten vorhanden ist
test_ids = test['Id'].values


In [112]:
# Erstellen des DataFrames mit 'Id' und 'SalePrice' (von XGBoost-Modell)
submission_xgboost = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': y_pred_xgboost
})

# Exportieren als CSV
submission_xgboost.to_csv('submission_xgboost.csv', index=False)

In [113]:
# Erstellen des DataFrames mit 'Id' und 'SalePrice' (von Keras NN-Modell)
submission_nn = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': y_pred_nn_final.flatten()  # Ensure the output is a flat array
})

# Exportieren als CSV
submission_nn.to_csv('submission_nn.csv', index=False)

NameError: name 'y_pred_nn_final' is not defined

In [114]:
# Vorhersagen mit dem Ridge-Modell
y_pred_ridge = best_ridge.predict(X_test_final)

# Erstellen des DataFrames mit 'Id' und 'SalePrice' (von Ridge-Modell)
submission_ridge = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': y_pred_ridge
})

# Exportieren als CSV
submission_ridge.to_csv('submission_ridge.csv', index=False)


/Users/linuszarse/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


In [115]:
# Vorhersagen mit dem Random Forest-Modell
y_pred_rf = best_rf.predict(X_test_final)

# Erstellen des DataFrames mit 'Id' und 'SalePrice' (von Random Forest-Modell)
submission_rf = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': y_pred_rf
})

# Exportieren als CSV
submission_rf.to_csv('submission_rf.csv', index=False)


/Users/linuszarse/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [116]:
# Vorhersagen mit dem Gradient Boosting-Modell
y_pred_gb = best_gb.predict(X_test_final)

# Erstellen des DataFrames mit 'Id' und 'SalePrice' (von Gradient Boosting-Modell)
submission_gb = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': y_pred_gb
})

# Exportieren als CSV
submission_gb.to_csv('submission_gb.csv', index=False)


/Users/linuszarse/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [117]:
# Vorhersagen mit dem XGBoost-Modell
y_pred_xgboost = best_xgb.predict(X_test_final)

# Erstellen des DataFrames mit 'Id' und 'SalePrice' (von XGBoost-Modell)
submission_xgboost = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': y_pred_xgboost
})

# Exportieren als CSV
submission_xgboost.to_csv('submission_xgboost.csv', index=False)
